# Llama3.cpp
### 1. build llama.cpp
1. `git clone https://github.com/ggerganov/llama.cpp`  
2. `cd llama.cpp`  
3. `make GGML_CUDA=1`  : gpu사용하려면 반드시 해당 옵션 넣어야 함
- Notes:
    For Q4_0_4_4 quantization type build, add the -DGGML_LLAMAFILE=OFF cmake option. For example, use cmake -B build -DGGML_LLAMAFILE=OFF.

    For faster compilation, add the -j argument to run multiple jobs in parallel. For example, cmake --build build --config Release -j 8 will run 8 jobs in parallel.

    For faster repeated compilation, install ccache.

    For debug builds, there are two cases:
4. install `llama-cpp-python` package at https://github.com/abetlen/llama-cpp-python.git
    - OS환경에 따른 환경변수 설정 옵션이 다르므로 확인 필요, 여기선 Linux로 가정하고 설명 진행 예정
    - 기본 설치는 cpu만 지원. 그러므로 원하는 특정 하드웨어에 대한 가속 옵션을 참고하여 설치해야함
        - CUDA 환경이므로, 
            To install with CUDA support, set the GGML_CUDA=on environment variable before installing:
            `CMAKE_ARGS="-DGGML_CUDA=on" pip install llama-cpp-python` 
            - `CMAKE_ARGS="-DGGML_CUDA=on" FORCE_CMAKE=1 pip install llama-cpp-python --force-reinstall --upgrade  --no-cache-dir` 


- deprecation: https://github.com/ggerganov/llama.cpp/blob/master/examples/deprecation-warning/README.md  

- quantise: https://github.com/ggerganov/llama.cpp/blob/master/examples/quantize/README.md  

- build: https://github.com/ggerganov/llama.cpp/blob/master/docs/build.md  

- main: https://github.com/ggerganov/llama.cpp/tree/master/examples/main  


### 2. LLM 모델 다운로드 
- https://dytis.tistory.com/72 참고
### 3. LLM 모델 convert (ggml, gguf 파일로 만들기)
llama.cpp 폴더에 들어가면 다음과 같은 파일들이 존재한다. 다음 중 내가 사용한 파일은 2개
you can use convert_hf_to_gguf.py with LLaMA 3 downloaded from Hugging Face.

1. convert_hf_to_gguf.py (다운 받은 모델을 cpp quantization 하여 gguf 파일로 만든다.)

2. convert-lora-to-ggml (lora로 학습시킨 adapter 모델을 ggml 파일로 만든다.)
### 4. Quantization the model
- 퀀타이제이션의 수가 작아질수록 성능(perplexity)가 안좋아짐(높아짐), 크기는 작아짐   
크기 <-> performace trade-off를 고려해서 골라야함 

# 나의 Llama3-8B-Instruct모델 Convert 과정
1. huggingface hub로부터 original폴더를 제외하고 다운로드
   - 모델의 checkpoint가 들어있는 original 폴더는 용량이 크고, llama.cpp에서는 
   - config.json, safetensor 파일 등을 사용하여 gguf로 변경할 수 있기 때문에 제외하고 다운로드
2. convert_fd_to_gguf 로 먼저 bf16 형식의 gguf파일으로 원본 모델을 변경: [fp32, fp16, bf16, q8_0] 중에서만 선택 가능 
3. 그 후, quantize를 사용해 4 bit quantization (Q4_K_M) 
4. main readme 참조) ./llama-cli 명령어로 Q4_K_M 모델 Run

### 1. download llama3-8b-instruct using huggingface hub

In [ ]:

from huggingface_hub import snapshot_download

snapshot_download(repo_id="meta-llama/Meta-Llama-3-8B-Instruct", local_dir="./models/llama_3_8b_instruct",
                  local_dir_use_symlinks=False, ignore_patterns=["original/*"],)


### 2. Convert ori model to bf16 type 
- `python convert_hf_to_gguf.py '/home/jetson/llamaR/Llama3-Playground/models/llama_3_8b_instruct' --outtype bf16` 

### 3. Quantization the bf16 model   
- `./llama-quantize /home/jetson/llamaR/Llama3-Playground/models/llama_3_8b_instruct/Llama_3_8B_Instruct-8.0B-BF16.gguf Q4_K_M`

-  해당 모델을 변환한 폴더 안에서  
   -  `ls -alh *.gguf | awk -v OFS='\t' '{print $5, $9}'`  
    으로 gguf 파일 크기 확인 가능   
   - `8.0G    Llama_3_8B_Instruct-8.0B-Q8_0.gguf`  
   - `4.6G	ggml-model-Q4_K_M.gguf`

### 4. Run the model 
- `- ./llama-cli -m /home/jetson/llamaR/Llama3-Playground/models/llama_3_8b_instruct/ggml-model-Q4_K_M.gguf \
-p "Hi" \
-ngl 128 \
-n 400 \
-t 16 \
--log-disable `
- 구체적인 프롬프트 및 답변 예시는 해당 repo의 **`robot_cmd.txt`** 참고

### robot_cmd.txt 내용 분석

- 실행 결과, -n 을 길게 줄수록 더 구체적인 답변을 내놓음
- 프롬프트를 더 자세히 준다면 원하는 답변을 Q4_k_M 모델에서도 이용 가능할 것으로 보임
- Qk_K_M 모델의 크기는 4.6GB임 

In [ ]:
# from llama_cpp import Llama


# model_path = '/home/jetson/llamaR/Llama3-Playground/models/llama_3_8b_instruct/Llama_3_8B_Instruct-8.0B-Q8_0.gguf'

# llama = Llama(model_path)